In [1]:
from pathlib import Path
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from collections import Counter

In [2]:
import gzip
import json
import io

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def unpack_dataset():
    ! wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Grocery_and_Gourmet_Food_5.json.gz
    #! wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Grocery_and_Gourmet_Food.csv
    ! mkdir data
    ! mv Gro* data

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [3]:
unpack_dataset()

--2020-06-24 20:37:39--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Grocery_and_Gourmet_Food_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146631394 (140M) [application/octet-stream]
Saving to: ‘Grocery_and_Gourmet_Food_5.json.gz’

Grocery_and_Gourmet 100%[===================>] 139.84M  11.2MB/s    in 11s     

2020-06-24 20:37:51 (12.5 MB/s) - ‘Grocery_and_Gourmet_Food_5.json.gz’ saved [146631394/146631394]



In [4]:
data = getDF('data/Grocery_and_Gourmet_Food_5.json.gz')

In [5]:
data.shape

(1143860, 12)

In [6]:
data = data.loc[(data.reviewTime.str.contains('2018'))] #| (data.reviewTime.str.contains('2016')) | (data.reviewTime.str.contains('2017')) | (data.reviewTime.str.contains('2018'))]

In [7]:
data.shape

(98253, 12)

In [8]:
# only use user, item, & ratings
df = data[['reviewerID', 'asin', 'overall']]
df.columns = ['user', 'item', 'rating']

In [9]:
del data

In [10]:
df.head()

,user,item,rating
50,A1U32SY2BN1I4T,9742356831,5.0
51,A2ULESH6GADUEW,9742356831,5.0
52,A1TI6UAU422P4Y,9742356831,3.0
53,A3VG446SHJH37O,9742356831,5.0
509,A2ZKFKG2PMJA3A,B00008RCN8,5.0


## Split Train/Validation

In [11]:
train, val = train_test_split(df, test_size=0.2)
train.shape, val.shape

((78602, 3), (19651, 3))

## Encode Places and Users

In [12]:
def encode_cols(train, val, cols):
    for col in cols:
        uniq = train[col].unique()
        col2idx = {val: i for i, val in enumerate(uniq)}
        train[col] = train[col].apply(lambda x: col2idx.get(x, -1))
        val[col] = val[col].apply(lambda x: col2idx.get(x, -1))
        val = val.loc[val[col] >= 0] 
    return train, val

In [13]:
train_enc, val_enc = encode_cols(train, val, ['user', 'item'])

In [14]:
len(train_enc.user.unique()), len(train_enc.item.unique())

(32860, 19067)

In [15]:
train_enc.head()

,user,item,rating
371813,0,0,5.0
1033729,1,1,5.0
1080193,2,2,5.0
1068894,3,3,5.0
992146,4,4,1.0


## Dataset and Data Loader

In [16]:
class AmazonDataset(Dataset):
    def __init__(self, X, y):
        self.user = X.user.values
        self.item = X.item.values
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.user[idx], self.item[idx], self.y[idx]

In [17]:
train_ds = AmazonDataset(train_enc[['item', 'user']], train_enc.rating.values)
valid_ds = AmazonDataset(val_enc[['item', 'user']], val_enc.rating.values)

In [18]:
batch_size = 5_000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [19]:
next(iter(train_dl))

[tensor([17005, 20167, 12152,  ..., 27147,    11,  1517]),
 tensor([18377, 11014,  6528,  ...,  5054,  1335, 10141]),
 tensor([5., 5., 4.,  ..., 5., 5., 5.], dtype=torch.float64)]

## Model

In [20]:
class MF_bias(nn.Module):
    def __init__(self, num_users, num_places, emb_size=100, bias=True):
        super(MF_bias, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.place_emb = nn.Embedding(num_places, emb_size)
        
        self.user_emb.weight.data.uniform_(0,0.05)
        self.place_emb.weight.data.uniform_(0,0.05)
        
        self.bias = bias
        if self.bias:
            self.user_bias = nn.Embedding(num_users, 1)
            self.place_bias = nn.Embedding(num_places, 1)
            
            self.user_bias.weight.data.uniform_(-0.01,0.01)
            self.place_emb.weight.data.uniform_(-0.01,0.01)
            
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, u, v):
        U = self.user_emb(u)
        V = self.place_emb(v)
        U, V = self.dropout(U), self.dropout(V)
        if self.bias:
            b_u = self.user_bias(u).squeeze()
            b_v = self.place_bias(v).squeeze()
            return (U*V).sum(1) +  b_u  + b_v
        return (U*V).sum(1)

## Training Functions

In [21]:
def cosine_segment(start_lr, end_lr, iterations):
    i = np.arange(iterations)
    c_i = 1 + np.cos(i*np.pi/iterations)
    return end_lr + (start_lr - end_lr)/2 *c_i

def get_cosine_triangular_lr(max_lr, iterations, div_start=5, div_end=5):
    min_start, min_end = max_lr/div_start, max_lr/div_end
    iter1 = int(0.3*iterations)
    iter2 = iterations - iter1
    segs = [cosine_segment(min_start, max_lr, iter1), cosine_segment(max_lr, min_end, iter2)]
    return np.concatenate(segs)

In [22]:
def set_learning_rate(optimizer, lr):
    """Changing learning rates without creating a new optimizer"""
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [23]:
def train_epocs(model, optimzer, train_dl, valid_dl, epochs=10, max_lr=0.01):
    iterations = epochs*len(train_dl)
    lrs = get_cosine_triangular_lr(max_lr, iterations)
    vals = []
    for epoch in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        i = 0
        for u, v, y in train_dl:
            set_learning_rate(optimizer, lrs[i])
            users = u.long().cuda()
            places = v.long().cuda()
            ratings = y.float().cuda()
            y_hat = model(users, places)
            loss = F.mse_loss(y_hat, ratings)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
            i += 1
        val_loss = val_metrics(model, valid_dl)
        vals.append(val_loss)
        if epoch % 10 == 0: 
            print(f"train loss {sum_loss/total:.3f} valid loss {val_loss:.3f}") #valid accuracy {testaccur:.5f}

In [24]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for u, v, y in valid_dl:
        users = u.long().cuda()
        places = v.long().cuda()
        ratings = y.float().cuda()
        y_hat = model(users, places)
        loss = F.mse_loss(y_hat, ratings)
        #y_pred = y_hat > 0
        #correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total #, correct/total

## Train

In [25]:
num_items, num_users = len(train_enc.item.unique()), len(train_enc.user.unique())
num_items, num_users

(19067, 32860)

In [31]:
model = MF_bias(num_users, num_items, emb_size=100, bias=False).cuda()

In [32]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)

In [33]:
train_epocs(model, optimizer, train_dl, valid_dl, max_lr=0.01, epochs=150)

train loss 20.125 valid loss 20.137
train loss 7.107 valid loss 6.881
train loss 1.910 valid loss 2.946
train loss 0.901 valid loss 2.226
train loss 0.670 valid loss 2.023
train loss 0.576 valid loss 1.953
train loss 0.528 valid loss 1.928
train loss 0.485 valid loss 1.917
train loss 0.451 valid loss 1.922
train loss 0.419 valid loss 1.920
train loss 0.397 valid loss 1.927
train loss 0.380 valid loss 1.939
train loss 0.371 valid loss 1.952
train loss 0.368 valid loss 1.970
train loss 0.365 valid loss 1.991


In [34]:
val_metrics(model, valid_dl)

2.0040587285937606